# Day 1: NumPy for Financial Arrays

## Week 1 - Python for Quantitative Finance

### 🎯 Learning Objectives
- Master NumPy array creation and manipulation for financial data
- Understand vectorization and why it matters for performance
- Apply broadcasting for efficient calculations
- Implement core financial computations using NumPy

### ⏱️ Time Allocation
- Theory review: 30 min
- Guided exercises: 90 min
- Practice problems: 60 min
- Interview prep: 30 min

---

**Author**: ML Quant Finance Mastery  
**Difficulty**: Foundation  
**Prerequisites**: Basic Python

## 1. Setup and Data Loading

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Download real market data using yfinance
tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5 years of data

print("📥 Downloading data from Yahoo Finance...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False, auto_adjust=True)

# Extract Close prices (auto_adjust=True gives adjusted prices as 'Close')
if isinstance(data.columns, pd.MultiIndex):
    prices_df = data['Close']
else:
    prices_df = data[['Close']]
    prices_df.columns = tickers

# Clean data
prices = prices_df.dropna()

print(f"✅ Data loaded: {prices.shape[0]} days, {len(tickers)} stocks")
print(f"📅 Date range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")
print(f"\n📊 Sample prices:")
prices.tail()

📥 Downloading data from Yahoo Finance...
✅ Data loaded: 1256 days, 5 stocks
📅 Date range: 2021-01-25 to 2026-01-23

📊 Sample prices:


Ticker,AAPL,GOOGL,GS,JPM,MSFT
Date,,,,,
2026-01-16,255.529999,330.000000,962.000000,312.470001,459.859985
2026-01-20,246.699997,322.000000,943.369995,302.739990,454.519989
2026-01-21,247.649994,328.380005,953.010010,302.040009,444.109985
2026-01-22,248.350006,330.540009,954.650024,303.630005,451.140015
2026-01-23,248.039993,327.929993,918.880005,297.720001,465.950012


## 2. NumPy Array Fundamentals

### 2.1 Creating Arrays from Financial Data

NumPy arrays are the foundation of quantitative finance in Python. They offer:
- **Homogeneous data types** (all elements same type → fast)
- **Contiguous memory** (cache-friendly → fast)
- **Vectorized operations** (no Python loops → fast)

In [2]:
price_array = prices.values
print(f"Array shape: {price_array.shape}")
print(f"Data type: {price_array.dtype}")
print(f"Memory size: {price_array.nbytes / 1024:.2f} KB")

print(f"\nRows = trading days ({price_array.shape[0]})")
print(f"Columns = stocks ({price_array.shape[1]})")

# Access patterns
print(f"\n📊 Single stock (AAPL) - all days: shape {price_array[:, 0].shape}")
print(f"📊 Single day (last) - all stocks: shape {price_array[-1, :].shape}")
print(f"📊 Last 5 days, first 3 stocks: shape {price_array[-5:, :3].shape}")

Array shape: (1256, 5)
Data type: float64
Memory size: 49.06 KB

Rows = trading days (1256)
Columns = stocks (5)

📊 Single stock (AAPL) - all days: shape (1256,)
📊 Single day (last) - all stocks: shape (5,)
📊 Last 5 days, first 3 stocks: shape (5, 3)


### 2.2 Vectorization: The Key to Performance

**Why does vectorization matter?**

In quant finance, you often need to:
- Calculate returns for 1000+ stocks × 5000+ days
- Run Monte Carlo with 100,000+ simulations
- Optimize portfolios in real-time

Python loops are too slow. NumPy vectorization runs optimized C code.

In [3]:
# Performance comparison: Loop vs Vectorized

# Task: Calculate simple returns for all stocks
n_iterations = 100

# METHOD 1: Python loops (slow)
def calculate_returns_loop(prices):
    n_days, n_stocks = prices.shape
    returns = np.zeros((n_days - 1, n_stocks))
    for i in range(1, n_days):
        for j in range(n_stocks):
            returns[i-1, j] = (prices[i, j] - prices[i-1, j]) / prices[i-1, j]
    return returns

# METHOD 2: NumPy vectorized (fast)
def calculate_returns_vectorized(prices):
    return (prices[1:] - prices[:-1]) / prices[:-1]

# Time both methods
start = time.perf_counter()
for _ in range(n_iterations):
    returns_loop = calculate_returns_loop(price_array)
loop_time = time.perf_counter() - start

start = time.perf_counter()
for _ in range(n_iterations):
    returns_vec = calculate_returns_vectorized(price_array)
vec_time = time.perf_counter() - start

print("⏱️ PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Loop method:       {loop_time:.4f} seconds")
print(f"Vectorized method: {vec_time:.4f} seconds")
print(f"Speedup:           {loop_time/vec_time:.1f}x faster!")

# Verify results are identical
print(f"\n✅ Results match: {np.allclose(returns_loop, returns_vec)}")

⏱️ PERFORMANCE COMPARISON
Loop method:       0.2421 seconds
Vectorized method: 0.0016 seconds
Speedup:           148.5x faster!

✅ Results match: True


## 3. Core Financial Calculations with NumPy

### 3.1 Returns: Simple vs Log

**Key Relationship**: $R = e^r - 1$ and $r = \ln(1+R)$

#### What is a Simple Return?

Simple return (also called arithmetic return) measures the percentage change in price:

$$R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1$$

**Properties:**
- **Intuitive**: A 10% return means you gained 10% of your investment
- **Multi-period**: Must be compounded → $(1+R_1)(1+R_2)...(1+R_n) - 1$
- **Portfolio returns**: Simple returns aggregate linearly across assets

#### What is a Log Return (Continuously Compounded Return)?

Log return is the natural logarithm of the price ratio:

$$r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) = \ln(P_t) - \ln(P_{t-1})$$

**Properties:**
- **Mathematically elegant**: Used in Black-Scholes, statistical models
- **Symmetric**: +10% and -10% have same magnitude
- **Time-additive**: Multi-period return = sum of log returns

#### When to Use Each?

| Use Case | Preferred Return |
|----------|------------------|
| Portfolio returns | Simple (aggregates across assets) |
| Time series analysis | Log (additive over time) |
| Statistical modeling | Log (better normality properties) |
| Reporting to clients | Simple (more intuitive) |
| VaR calculations | Log (for parametric VaR) |

In [4]:
# Simple returns: R_t = (P_t - P_{t-1}) / P_{t-1}
simple_returns = (price_array[1:] - price_array[:-1]) / price_array[:-1]

# Log returns: r_t = ln(P_t / P_{t-1})
log_returns = np.log(price_array[1:] / price_array[:-1])

print("📊 RETURNS COMPARISON")
print("=" * 60)
print(f"\nSimple Returns (first 5 days, AAPL):")
print(simple_returns[:5, 0].round(4))

print(f"\nLog Returns (first 5 days, AAPL):")
print(log_returns[:5, 0].round(4))

# Key difference: additivity
print("\n" + "=" * 60)
print("📐 KEY PROPERTY: Log returns are ADDITIVE")
print("=" * 60)

# Multi-period return calculation
n_days = 20  # Calculate 20-day return

# Simple returns: must compound (multiply)
simple_20d = np.prod(1 + simple_returns[:n_days, 0]) - 1

# Log returns: just add
log_20d = np.sum(log_returns[:n_days, 0])

print(f"\n20-day return (AAPL):")
print(f"  Simple (compounded): {simple_20d:.4f} ({simple_20d*100:.2f}%)")
print(f"  Log (summed):        {log_20d:.4f} ({log_20d*100:.2f}%)")
print(f"  Log → Simple:        {np.exp(log_20d) - 1:.4f}")  # Convert back

📊 RETURNS COMPARISON

Simple Returns (first 5 days, AAPL):
[ 0.0017 -0.0077 -0.035  -0.0374  0.0165]

Log Returns (first 5 days, AAPL):
[ 0.0017 -0.0077 -0.0356 -0.0381  0.0164]

📐 KEY PROPERTY: Log returns are ADDITIVE

20-day return (AAPL):
  Simple (compounded): -0.1181 (-11.81%)
  Log (summed):        -0.1256 (-12.56%)
  Log → Simple:        -0.1181


### 3.2 Volatility and Risk Metrics

#### What is Volatility?

Volatility measures the dispersion of returns - how much the price "jumps around".

$$\sigma = \sqrt{\frac{1}{N-1}\sum_{i=1}^{N}(r_i - \bar{r})^2}$$

**Key Points:**
- **Daily volatility**: Standard deviation of daily returns
- **Annualized volatility**: $\sigma_{annual} = \sigma_{daily} \times \sqrt{252}$ (assuming 252 trading days)
- Higher volatility = more risk (and opportunity)

#### Key Risk Metrics in Quant Trading

| Metric | Formula | What It Measures |
|--------|---------|------------------|
| **Sharpe Ratio** | $(R_p - R_f) / \sigma_p$ | Risk-adjusted return |
| **Max Drawdown** | Max peak-to-trough decline | Worst-case scenario |
| **VaR (95%)** | 5th percentile of returns | Potential daily loss |
| **Sortino Ratio** | $(R_p - R_f) / \sigma_{down}$ | Downside risk-adjusted |

#### In Quant Trading & Risk Management

- VaR is used for regulatory capital requirements (Basel III, Solvency II)
- Max drawdown helps set stop-losses and risk limits
- Sharpe > 1 is generally considered good risk-adjusted performance
- Volatility is used for position sizing (lower vol = larger position)

In [5]:
# Calculate key risk metrics for each stock
TRADING_DAYS = 252
RISK_FREE_RATE = 0.05  # 5% annual

# Daily metrics
daily_mean = np.mean(simple_returns, axis=0)
daily_std = np.std(simple_returns, axis=0, ddof=1)  # ddof=1 for sample std

# Annualized metrics
annual_return = daily_mean * TRADING_DAYS
annual_vol = daily_std * np.sqrt(TRADING_DAYS)

# Sharpe Ratio
sharpe_ratio = (annual_return - RISK_FREE_RATE) / annual_vol

# Max Drawdown
def calculate_max_drawdown(prices):
    """Calculate maximum drawdown for each column."""
    cummax = np.maximum.accumulate(prices, axis=0)
    drawdown = (prices - cummax) / cummax
    return np.min(drawdown, axis=0)

max_dd = calculate_max_drawdown(price_array)

# Display results
print("📊 RISK METRICS SUMMARY")
print("=" * 70)
print(f"\n{'Metric':<20} " + " ".join(f"{t:>10}" for t in tickers))
print("-" * 70)
print(f"{'Ann. Return':<20} " + " ".join(f"{r*100:>9.2f}%" for r in annual_return))
print(f"{'Ann. Volatility':<20} " + " ".join(f"{v*100:>9.2f}%" for v in annual_vol))
print(f"{'Sharpe Ratio':<20} " + " ".join(f"{s:>10.2f}" for s in sharpe_ratio))
print(f"{'Max Drawdown':<20} " + " ".join(f"{d*100:>9.2f}%" for d in max_dd))

📊 RISK METRICS SUMMARY

Metric                     AAPL       MSFT      GOOGL        JPM         GS
----------------------------------------------------------------------
Ann. Return              15.42%     29.90%     29.76%     21.82%     18.34%
Ann. Volatility          27.67%     31.02%     27.33%     24.29%     25.70%
Sharpe Ratio               0.38       0.80       0.91       0.69       0.52
Max Drawdown            -33.36%    -44.32%    -32.84%    -38.77%    -37.15%


## 4. Broadcasting: Efficient Cross-Sectional Calculations

Broadcasting allows operations between arrays of different shapes. This is essential for:
- Demeaning returns (subtract mean from each stock)
- Standardizing data (z-scores)
- Portfolio calculations

In [6]:
# Broadcasting example: Z-score normalization

# Step 1: Calculate mean and std for each stock (across time)
means = np.mean(simple_returns, axis=0)  # Shape: (5,)
stds = np.std(simple_returns, axis=0)    # Shape: (5,)

print(f"Returns shape:    {simple_returns.shape}")  # (1770, 5)
print(f"Means shape:      {means.shape}")           # (5,)
print(f"Stds shape:       {stds.shape}")            # (5,)

# Step 2: Broadcasting automatically aligns dimensions
# (1770, 5) - (5,) → broadcasts to (1770, 5) - (1770, 5)
z_scores = (simple_returns - means) / stds

print(f"Z-scores shape:   {z_scores.shape}")

# Verify z-scores have mean ≈ 0 and std ≈ 1
print(f"\n✅ Z-score verification:")
print(f"   Means: {np.mean(z_scores, axis=0).round(10)}")  # Should be ~0
print(f"   Stds:  {np.std(z_scores, axis=0).round(4)}")    # Should be ~1

Returns shape:    (1255, 5)
Means shape:      (5,)
Stds shape:       (5,)
Z-scores shape:   (1255, 5)

✅ Z-score verification:
   Means: [0. 0. 0. 0. 0.]
   Stds:  [1. 1. 1. 1. 1.]


## 5. Correlation and Covariance Matrices

### What is Covariance?

Covariance measures how two assets move together:

$$Cov(X,Y) = E[(X - \mu_X)(Y - \mu_Y)]$$

**Interpretation:**
- **Positive covariance**: Assets move in the same direction
- **Negative covariance**: Assets move in opposite directions
- **Zero covariance**: No linear relationship
- **Problem**: Scale-dependent (hard to interpret the magnitude)

### The Covariance Matrix ($\Sigma$)

An $n \times n$ matrix where:
- **Diagonal**: Variance of each asset ($\sigma_i^2$)
- **Off-diagonal**: Covariance between asset pairs ($Cov(i,j)$)

### What is Correlation?

Correlation is standardized covariance (ranges from -1 to +1):

$$\rho_{X,Y} = \frac{Cov(X,Y)}{\sigma_X \sigma_Y}$$

**Interpretation:**
- **ρ = +1**: Perfect positive correlation
- **ρ = -1**: Perfect negative correlation
- **ρ = 0**: No linear relationship

### Why These Matter in Quant Trading & Risk Management

| Application | How It's Used |
|-------------|---------------|
| **Portfolio Diversification** | Low correlation → better diversification |
| **Risk Management** | Correlation spikes during crises ("correlations go to 1") |
| **Pairs Trading** | High correlation stocks for mean-reversion strategies |
| **Hedging** | Find negatively correlated assets to hedge exposure |
| **Factor Models** | Covariance decomposition into systematic factors |
| **VaR Calculation** | Portfolio VaR requires full covariance matrix |

In [7]:
# Correlation matrix
corr_matrix = np.corrcoef(simple_returns.T)  # Transpose: stocks as rows

# Covariance matrix (annualized)
cov_matrix = np.cov(simple_returns.T) * TRADING_DAYS

print("📊 CORRELATION MATRIX")
print("=" * 60)
print(f"\n{'':>10}" + "".join(f"{t:>10}" for t in tickers))
for i, ticker in enumerate(tickers):
    print(f"{ticker:>10}" + "".join(f"{corr_matrix[i,j]:>10.3f}" for j in range(len(tickers))))

print("\n\n📊 ANNUALIZED COVARIANCE MATRIX")
print("=" * 60)
print(f"\n{'':>10}" + "".join(f"{t:>10}" for t in tickers))
for i, ticker in enumerate(tickers):
    print(f"{ticker:>10}" + "".join(f"{cov_matrix[i,j]:>10.4f}" for j in range(len(tickers))))

# Key insight: diagonal = variance, off-diagonal = covariance
print("\n📐 Key insight:")
print(f"   Diagonal elements = Variance (volatility²)")
print(f"   AAPL variance: {cov_matrix[0,0]:.4f}, volatility: {np.sqrt(cov_matrix[0,0]):.4f}")

📊 CORRELATION MATRIX

                AAPL      MSFT     GOOGL       JPM        GS
      AAPL     1.000     0.577     0.407     0.355     0.627
      MSFT     0.577     1.000     0.381     0.325     0.635
     GOOGL     0.407     0.381     1.000     0.771     0.373
       JPM     0.355     0.325     0.771     1.000     0.317
        GS     0.627     0.635     0.373     0.317     1.000


📊 ANNUALIZED COVARIANCE MATRIX

                AAPL      MSFT     GOOGL       JPM        GS
      AAPL    0.0766    0.0495    0.0308    0.0239    0.0446
      MSFT    0.0495    0.0962    0.0323    0.0245    0.0506
     GOOGL    0.0308    0.0323    0.0747    0.0512    0.0262
       JPM    0.0239    0.0245    0.0512    0.0590    0.0198
        GS    0.0446    0.0506    0.0262    0.0198    0.0660

📐 Key insight:
   Diagonal elements = Variance (volatility²)
   AAPL variance: 0.0766, volatility: 0.2767


## 6. Portfolio Calculations

### 6.1 Portfolio Return and Risk

For a portfolio with weights $w$, returns $r$, and covariance matrix $\Sigma$:

$$R_p = w^T r \quad \text{(Portfolio Return)}$$
$$\sigma_p^2 = w^T \Sigma w \quad \text{(Portfolio Variance)}$$

In [8]:
# Define portfolio weights (equal-weighted)
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Portfolio return
portfolio_return = np.dot(weights, annual_return)

# Portfolio variance using matrix multiplication
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
portfolio_volatility = np.sqrt(portfolio_variance)

# Portfolio Sharpe ratio
portfolio_sharpe = (portfolio_return - RISK_FREE_RATE) / portfolio_volatility

print("📊 EQUAL-WEIGHT PORTFOLIO METRICS")
print("=" * 50)
print(f"\nWeights: {dict(zip(tickers, weights))}")
print(f"\nExpected Annual Return: {portfolio_return*100:.2f}%")
print(f"Portfolio Volatility:   {portfolio_volatility*100:.2f}%")
print(f"Portfolio Sharpe Ratio: {portfolio_sharpe:.2f}")

# Compare to individual stocks
print(f"\n📊 DIVERSIFICATION BENEFIT")
print("-" * 50)
avg_individual_vol = np.mean(annual_vol)
print(f"Average individual volatility: {avg_individual_vol*100:.2f}%")
print(f"Portfolio volatility:          {portfolio_volatility*100:.2f}%")
print(f"Risk reduction:                {(1 - portfolio_volatility/avg_individual_vol)*100:.1f}%")

📊 EQUAL-WEIGHT PORTFOLIO METRICS

Weights: {'AAPL': np.float64(0.2), 'MSFT': np.float64(0.2), 'GOOGL': np.float64(0.2), 'JPM': np.float64(0.2), 'GS': np.float64(0.2)}

Expected Annual Return: 23.05%
Portfolio Volatility:   20.78%
Portfolio Sharpe Ratio: 0.87

📊 DIVERSIFICATION BENEFIT
--------------------------------------------------
Average individual volatility: 27.20%
Portfolio volatility:          20.78%
Risk reduction:                23.6%


## 7. Practice Problems

### Problem 1: Rolling Volatility
Calculate 20-day rolling volatility for AAPL using NumPy (no pandas rolling!).

In [9]:
# SOLUTION: Rolling volatility using stride tricks

def rolling_volatility(returns: np.ndarray, window: int) -> np.ndarray:
    """
    Calculate rolling volatility using NumPy stride tricks.
    
    This is faster than looping but more complex.
    In practice, pandas rolling is preferred for readability.
    """
    n = len(returns)
    
    # Method 1: Simple loop (baseline)
    # rolling_std = np.array([returns[i:i+window].std(ddof=1) 
    #                         for i in range(n - window + 1)])
    
    # Method 2: Stride tricks (advanced, faster)
    from numpy.lib.stride_tricks import sliding_window_view
    windows = sliding_window_view(returns, window)
    rolling_std = np.std(windows, axis=1, ddof=1)
    
    return rolling_std * np.sqrt(TRADING_DAYS)  # Annualize

# Calculate for AAPL
aapl_returns = simple_returns[:, 0]
rolling_vol = rolling_volatility(aapl_returns, window=20)

print(f"📊 Rolling 20-day Volatility (AAPL)")
print(f"   Shape: {rolling_vol.shape}")
print(f"   Min:   {rolling_vol.min()*100:.2f}%")
print(f"   Max:   {rolling_vol.max()*100:.2f}%")
print(f"   Mean:  {rolling_vol.mean()*100:.2f}%")
print(f"\n   Last 5 values: {rolling_vol[-5:].round(4)}")

📊 Rolling 20-day Volatility (AAPL)
   Shape: (1236,)
   Min:   9.46%
   Max:   81.81%
   Mean:  25.64%

   Last 5 values: [0.1055 0.1523 0.1509 0.1522 0.1486]


### Problem 2: Monte Carlo Simulation
Simulate 10,000 possible 1-year price paths for AAPL assuming geometric Brownian motion.

In [10]:
# Monte Carlo simulation using Geometric Brownian Motion
# dS = μS dt + σS dW

# Parameters
S0 = price_array[-1, 0]  # Current AAPL price
mu = annual_return[0]     # Drift (expected return)
sigma = annual_vol[0]     # Volatility
T = 1.0                   # Time horizon (1 year)
n_steps = 252             # Daily steps
n_simulations = 10000

# Time step
dt = T / n_steps

# Generate random shocks (all at once for efficiency)
np.random.seed(42)
Z = np.random.standard_normal((n_simulations, n_steps))

# Simulate paths using vectorized operations
# S(t+dt) = S(t) * exp((μ - σ²/2)dt + σ√dt * Z)
drift = (mu - 0.5 * sigma**2) * dt
diffusion = sigma * np.sqrt(dt) * Z

# Cumulative sum of log returns
log_returns_sim = drift + diffusion
cum_log_returns = np.cumsum(log_returns_sim, axis=1)

# Convert to prices
price_paths = S0 * np.exp(cum_log_returns)

# Add initial price
price_paths = np.column_stack([np.full(n_simulations, S0), price_paths])

print(f"📊 MONTE CARLO SIMULATION RESULTS")
print(f"=" * 50)
print(f"Initial price: ${S0:.2f}")
print(f"Simulations:   {n_simulations:,}")
print(f"Time horizon:  {T} year ({n_steps} days)")
print(f"\n📈 Final Price Distribution:")
final_prices = price_paths[:, -1]
print(f"   Mean:   ${np.mean(final_prices):.2f}")
print(f"   Median: ${np.median(final_prices):.2f}")
print(f"   5th %:  ${np.percentile(final_prices, 5):.2f}")
print(f"   95th %: ${np.percentile(final_prices, 95):.2f}")
print(f"\n📉 Value at Risk (95%):")
print(f"   VaR: ${S0 - np.percentile(final_prices, 5):.2f} ({(1 - np.percentile(final_prices, 5)/S0)*100:.1f}%)")

📊 MONTE CARLO SIMULATION RESULTS
Initial price: $248.04
Simulations:   10,000
Time horizon:  1.0 year (252 days)

📈 Final Price Distribution:
   Mean:   $288.59
   Median: $277.79
   5th %:  $175.50
   95th %: $436.35

📉 Value at Risk (95%):
   VaR: $72.54 (29.2%)


## 8. Interview Practice Questions

### Question 1 (Jane Street style)
*You have daily returns for 100 stocks over 5 years. How would you efficiently compute the correlation between every pair of stocks?*

In [11]:
# SOLUTION to Interview Question 1

# Simulate the data
n_stocks = 100
n_days = 252 * 5  # 5 years
returns_large = np.random.randn(n_days, n_stocks) * 0.02  # Simulated returns

# Efficient correlation computation
start = time.perf_counter()
corr_full = np.corrcoef(returns_large.T)  # np.corrcoef expects features as rows
elapsed = time.perf_counter() - start

print(f"📊 INTERVIEW ANSWER")
print("=" * 50)
print(f"Input: {n_stocks} stocks × {n_days} days")
print(f"Output: {n_stocks}×{n_stocks} correlation matrix")
print(f"Unique pairs: {n_stocks * (n_stocks - 1) // 2:,}")
print(f"Computation time: {elapsed*1000:.2f} ms")

print(f"\n💡 Key insight: np.corrcoef() uses efficient linear algebra")
print(f"   Under the hood: standardize → matrix multiply → efficient BLAS")
print(f"\n   corr = np.corrcoef(returns.T)  # That's it!")

📊 INTERVIEW ANSWER
Input: 100 stocks × 1260 days
Output: 100×100 correlation matrix
Unique pairs: 4,950
Computation time: 1.16 ms

💡 Key insight: np.corrcoef() uses efficient linear algebra
   Under the hood: standardize → matrix multiply → efficient BLAS

   corr = np.corrcoef(returns.T)  # That's it!


## 9. Summary & Key Takeaways

### ✅ What You Learned Today

1. **NumPy arrays** are the foundation for efficient financial calculations
2. **Vectorization** provides 100x+ speedup over Python loops
3. **Broadcasting** enables elegant cross-sectional calculations
4. **Returns**: Simple for portfolios, Log for time series
5. **Risk metrics**: Volatility, Sharpe, Max Drawdown
6. **Portfolio math**: $R_p = w^T r$, $\sigma_p^2 = w^T \Sigma w$

### 🎯 Interview Tips

- Always use vectorized operations
- Know the difference between simple and log returns
- Understand correlation vs covariance
- Be comfortable with matrix notation for portfolio calculations

### 📚 Tomorrow's Preview

**Day 2: Pandas TimeSeries & Point-in-Time Data**
- DatetimeIndex mastery
- Resampling and alignment
- Look-ahead bias prevention

## 🔴 PROS & CONS: NumPy for Financial Computing

### ✅ PROS (Why NumPy is Industry Standard)

| Advantage | Details | Real-World Impact |
|-----------|---------|-------------------|
| **Speed** | 100-1000x faster than Python loops | Essential for HFT, real-time pricing |
| **Memory Efficient** | Contiguous arrays, no Python overhead | Handle millions of data points |
| **Broadcasting** | Automatic alignment of different-sized arrays | Clean, readable code |
| **Ecosystem** | Pandas, SciPy, scikit-learn all built on NumPy | Seamless integration |
| **Battle-Tested** | Used by every major hedge fund, bank | Reliable, well-documented |

### ❌ CONS (Limitations to Know)

| Limitation | Details | Workaround |
|------------|---------|------------|
| **No Labels** | Pure arrays, no column names or dates | Use Pandas for labeled data |
| **Homogeneous** | All elements must be same type | Use structured arrays if needed |
| **In-Memory** | Must fit in RAM | Use Dask or Vaex for big data |
| **No GPU** | CPU only by default | Use CuPy or JAX for GPU |
| **Learning Curve** | Broadcasting can be confusing | Practice with simple examples first |

### 🎯 Real-World Usage

**WHERE IT'S USED:**
- ✅ Backtesting engines (Zipline, Backtrader)
- ✅ Risk systems (VaR, Monte Carlo)
- ✅ Portfolio optimization (mean-variance)
- ✅ Option pricing (Black-Scholes, binomial trees)
- ✅ Signal generation in trading systems

**NOT THEORY - THIS IS PRODUCTION CODE:**
Every quant desk uses NumPy. When you write `np.corrcoef()`, you're using the same code as Two Sigma, Citadel, and Renaissance.

## 🚀 TODAY'S TRADING SIGNAL (Real-World Application)

Using the methods from this notebook, let's generate an actionable trading signal for today.

In [12]:
# =============================================================================
# TODAY'S TRADING SIGNAL - Using NumPy Analysis
# =============================================================================

print("=" * 70)
print("📊 TODAY'S TRADING ANALYSIS - Based on NumPy Metrics")
print("=" * 70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print()

# Get the most recent data
latest_prices = prices.iloc[-1]
print("📈 CURRENT PRICES:")
for ticker in tickers:
    print(f"   {ticker}: ${latest_prices[ticker]:.2f}")

# Calculate recent momentum (20-day return)
momentum_20d = (prices.iloc[-1] / prices.iloc[-20] - 1) * 100
# Calculate recent volatility (20-day)
recent_vol = simple_returns[-20:].std(axis=0) * np.sqrt(252) * 100

# Sharpe ratio based on last 60 days
recent_sharpe = (simple_returns[-60:].mean(axis=0) * 252 - 0.05) / (simple_returns[-60:].std(axis=0) * np.sqrt(252))

print("\n📊 20-DAY MOMENTUM:")
for i, ticker in enumerate(tickers):
    direction = "🟢" if momentum_20d[ticker] > 0 else "🔴"
    print(f"   {direction} {ticker}: {momentum_20d[ticker]:+.2f}%")

print("\n📊 CURRENT VOLATILITY (Annualized):")
for i, ticker in enumerate(tickers):
    vol_level = "🔴 HIGH" if recent_vol[i] > 30 else "🟡 MEDIUM" if recent_vol[i] > 20 else "🟢 LOW"
    print(f"   {ticker}: {recent_vol[i]:.1f}% ({vol_level})")

print("\n📊 60-DAY SHARPE RATIO:")
for i, ticker in enumerate(tickers):
    sharpe_level = "🟢 STRONG" if recent_sharpe[i] > 1 else "🟡 MODERATE" if recent_sharpe[i] > 0 else "🔴 WEAK"
    print(f"   {ticker}: {recent_sharpe[i]:.2f} ({sharpe_level})")

# Generate trading signals
print("\n" + "=" * 70)
print("🎯 TRADING SIGNALS FOR TODAY")
print("=" * 70)

for i, ticker in enumerate(tickers):
    print(f"\n{'='*30} {ticker} {'='*30}")
    
    # Simple rule-based signal
    momentum_score = 1 if momentum_20d[ticker] > 5 else -1 if momentum_20d[ticker] < -5 else 0
    vol_score = -1 if recent_vol[i] > 35 else 0  # Penalize high vol
    sharpe_score = 1 if recent_sharpe[i] > 1 else -1 if recent_sharpe[i] < 0 else 0
    
    total_score = momentum_score + vol_score + sharpe_score
    
    if total_score >= 2:
        signal = "🟢 STRONG BUY"
        action = "Consider buying shares or CALL options"
    elif total_score == 1:
        signal = "🟡 WEAK BUY"  
        action = "Consider small position or wait for pullback"
    elif total_score == 0:
        signal = "⚪ HOLD/NEUTRAL"
        action = "No clear edge - stay on sidelines or hold existing"
    elif total_score == -1:
        signal = "🟠 WEAK SELL"
        action = "Consider trimming position or PUT spreads"
    else:
        signal = "🔴 STRONG SELL"
        action = "Consider selling or protective PUTs"
    
    print(f"   Signal: {signal}")
    print(f"   Action: {action}")
    print(f"   Reasoning: Momentum={momentum_20d[ticker]:+.1f}%, Vol={recent_vol[i]:.1f}%, Sharpe={recent_sharpe[i]:.2f}")

# Portfolio recommendation
print("\n" + "=" * 70)
print("📊 PORTFOLIO RECOMMENDATION (Equal Weight)")
print("=" * 70)
print(f"   Expected Annual Return: {portfolio_return*100:.1f}%")
print(f"   Portfolio Volatility: {portfolio_volatility*100:.1f}%")
print(f"   Sharpe Ratio: {portfolio_sharpe:.2f}")

if portfolio_sharpe > 1:
    print("\n   ✅ RECOMMENDATION: Attractive risk-adjusted returns. Consider allocation.")
elif portfolio_sharpe > 0.5:
    print("\n   🟡 RECOMMENDATION: Moderate opportunity. Position size conservatively.")
else:
    print("\n   ⚠️ RECOMMENDATION: Poor risk-adjusted returns. Wait for better setup.")

print("\n" + "=" * 70)
print("⚠️ DISCLAIMER: This is educational analysis, not financial advice.")
print("   Always do your own research and consider your risk tolerance.")
print("=" * 70)

📊 TODAY'S TRADING ANALYSIS - Based on NumPy Metrics
Analysis Date: 2026-01-24 09:45

📈 CURRENT PRICES:
   AAPL: $248.04
   MSFT: $465.95
   GOOGL: $327.93
   JPM: $297.72
   GS: $918.88

📊 20-DAY MOMENTUM:
   🔴 AAPL: -9.41%
   🔴 MSFT: -4.52%
   🟢 GOOGL: +4.41%
   🔴 JPM: -9.15%
   🟢 GS: +0.89%

📊 CURRENT VOLATILITY (Annualized):
   AAPL: 14.5% (🟢 LOW)
   MSFT: 17.1% (🟢 LOW)
   GOOGL: 32.5% (🔴 HIGH)
   JPM: 24.9% (🟡 MEDIUM)
   GS: 21.6% (🟡 MEDIUM)

📊 60-DAY SHARPE RATIO:
   AAPL: -2.60 (🔴 WEAK)
   MSFT: 2.91 (🟢 STRONG)
   GOOGL: 2.35 (🟢 STRONG)
   JPM: -0.38 (🔴 WEAK)
   GS: -2.68 (🔴 WEAK)

🎯 TRADING SIGNALS FOR TODAY

============================== AAPL ==============================
   Signal: 🔴 STRONG SELL
   Action: Consider selling or protective PUTs
   Reasoning: Momentum=-9.4%, Vol=14.5%, Sharpe=-2.60

============================== MSFT ==============================
   Signal: 🟡 WEAK BUY
   Action: Consider small position or wait for pullback
   Reasoning: Momentum=-4.5%, Vol=17.